In [1]:
from collections import Counter
from itertools import islice

from Bio import SeqIO
import numpy as np
import pandas as pd

In [46]:
def load_data_from_file(fpath):
    data = [record for record in SeqIO.parse(fpath, 'fasta')]
    strands = [d.seq._data for d in data]
    char_arr = np.empty(shape=(len(strands), len(strands[0])), dtype=str)
    for ix, strand in enumerate(strands):
        chars = [char for char in strand]
        char_arr[ix, :] = chars
    return char_arr


def random_start_motif(strand, k=20):
    """Picks random starting motifs from the provided DNA strings"""
    N = len(strand)
    poss_starts = len(strand) - k + 1
    start = np.random.choice(poss_starts)
    motif = strand[start:start+k]
    assert len(motif) == k, "Motif is wrong length"
    return motif


def random_start(char_arr, k=20):
    motifs = np.empty(shape=(len(char_arr), k), dtype=str)
    for ix, strand in enumerate(char_arr):
        motifs[ix, :] = random_start_motif(strand, k)
    return motifs


def remove_motif(motifs, strand_ix):
    msk = np.ones(len(motifs), dtype=bool)
    msk[strand_ix] = False
    return motifs[msk]
    
    
def profile(motifs):
    counts = pd.DataFrame(index=["A", "C", "G", "T"], columns=range(len(motifs[0])))
    # Laplace correction makes every count at least 1
    counts = counts.fillna(1)
    for col_ix in range(len(motifs[0])):
        col = motifs[:, col_ix]
        c = Counter(col)
        for nuc, cnt in c.items():
            counts.loc[nuc, col_ix] += cnt
    counts = counts.apply(lambda col: col/col.sum()) # normalized
    return counts
        
    
def profile_motif(motif, profile):
    profile_score = 1
    for ix, nuc in enumerate(motif):
        profile_score *= profile.loc[nuc, ix]
    return profile_score


def window(seq, k):
    it = iter(seq)
    result = tuple(islice(it, k))
    if len(result) == k:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result
    
    
def profile_strand(strand, profile):
    profile_scores = []
    k = profile.shape[1]
    for motif in window(strand, k):
        profile_scores.append(profile_motif(motif, profile))
    ttl = sum(profile_scores)
    return [score/ttl for score in profile_scores]


def sample_new_motif(strand, scores, k=20):
    # Each score corresponds to the starting index of that motif
    start_ix = np.random.choice(range(len(scores)), p=scores)
    new_motif = strand[start_ix:start_ix+k]
    assert len(new_motif) == k, "new motif is wrong length" # error checking
    return new_motif


def score_motifs(motifs):
    score = 0
    for col in range(motifs.shape[1]):
        most_freq_nuc, cnt_most_freq = Counter(motifs[:,col]).most_common(1)[0]
        # score for a col is the number of nucleotides that are NOT
        # the most frequent
        score += (len(motifs[:,col]) - cnt_most_freq)
    return score
    
    
def gibbs(dna, k, N):    
    motifs = random_start(dna, k)
    best_motifs = motifs
    best_score = score_motifs(motifs)
    num_strands = dna.shape[0]
    for _ in range(N):
        strand_ix = np.random.choice(num_strands)
        with_motif_removed = remove_motif(motifs, strand_ix)
        prof = profile(with_motif_removed)
        strand_probs = profile_strand(dna[strand_ix, :], prof)
        new_motif = sample_new_motif(dna[strand_ix, :], strand_probs, k)
        motifs[strand_ix, :] = new_motif
        if score_motifs(motifs) < best_score:
            best_motifs = motifs
            best_score = score_motifs(motifs)
    return best_score, best_motifs

In [47]:
dna = load_data_from_file('upstream_250.txt')
results = []
for it in range(100): # try 1k random starts
    if it % 100 == 0:
        print("On iteration ", it)
    result = gibbs(dna, k=20, N=100)
    results.append(result)

On iteration  0


In [56]:
results.sort(key=lambda tup: tup[0])

In [57]:
results[0][0]

322

In [58]:
results[0][1]

array([['C', 'A', 'G', 'C', 'C', 'C', 'C', 'A', 'A', 'G', 'G', 'C', 'C',
        'G', 'A', 'A', 'C', 'G', 'A', 'C'],
       ['C', 'G', 'A', 'C', 'C', 'A', 'C', 'C', 'G', 'C', 'G', 'C', 'C',
        'G', 'G', 'A', 'G', 'C', 'C', 'C'],
       ['A', 'G', 'G', 'G', 'C', 'T', 'C', 'C', 'G', 'G', 'C', 'G', 'C',
        'G', 'G', 'T', 'G', 'G', 'T', 'C'],
       ['C', 'G', 'G', 'C', 'C', 'C', 'T', 'G', 'T', 'C', 'C', 'G', 'C',
        'G', 'T', 'C', 'C', 'G', 'T', 'G'],
       ['C', 'G', 'A', 'A', 'C', 'C', 'C', 'G', 'A', 'G', 'A', 'T', 'C',
        'G', 'T', 'C', 'A', 'C', 'C', 'G'],
       ['C', 'C', 'G', 'C', 'C', 'G', 'T', 'T', 'G', 'C', 'G', 'C', 'C',
        'G', 'G', 'G', 'T', 'G', 'C', 'G'],
       ['C', 'G', 'G', 'C', 'C', 'G', 'A', 'C', 'G', 'A', 'G', 'C', 'G',
        'G', 'G', 'C', 'G', 'C', 'A', 'G'],
       ['A', 'G', 'C', 'G', 'C', 'G', 'C', 'G', 'G', 'G', 'C', 'C', 'C',
        'G', 'G', 'C', 'C', 'G', 'C', 'C'],
       ['C', 'G', 'A', 'G', 'G', 'A', 'G', 'C', 'A', 'C', 'A', '